# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import pickle
import re
import warnings
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///MESSAGE_DATABASE.db')
df = pd.read_sql("SELECT * FROM MESSAGES", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [3]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [4]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """
    inputs:
    Raw text
       
    Returns:
    List of cleaned words / tokens
    """
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # normalization word tokens and remove stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    normlized = [normlizer.stem(word) for word in tokens if word not in stop_words]
    
    return normlized

In [7]:
# Test tokenize function
print('Raw text:', X[11])
print('Tokenized:', tokenize(X[11]))

Raw text: I am in Petionville. I need more information regarding 4636
Tokenized: ['petionvil', 'need', 'inform', 'regard', '4636']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

np.random.seed(2021)
pipe.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_train_preds = pipe.predict(X_train)
y_test_preds = pipe.predict(X_test)

In [11]:
# Function to get test metrics given actual and predicted labels
def get_metrics(y_test, y_pred):
    
    # Get dataframe of f_score, precision, recall for each category
    results_df = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    num = 0
    for cat in y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,num], average='weighted')
        results_df.set_value(num+1, 'Category', cat)
        results_df.set_value(num+1, 'f_score', f_score)
        results_df.set_value(num+1, 'precision', precision)
        results_df.set_value(num+1, 'recall', recall)
        num += 1
        
    # Get averaged scores
    scores = {}
    scores['Average f_score'] =  results_df['f_score'].mean()
    scores['Average precision'] = results_df['precision'].mean()
    scores['Average recall'] = results_df['recall'].mean()

    return results_df, scores

test_metrics_df, test_avg_metrics = get_metrics(y_test = Y_test, y_pred = y_test_preds)
train_metrics_df, train_avg_metrics = get_metrics(y_test = Y_train, y_pred = y_train_preds)

In [12]:
print('Train scores: ', train_avg_metrics)
print('Test scores: ', test_avg_metrics)

Train scores:  {'Average f_score': 0.99261430842397835, 'Average precision': 0.99292302952106937, 'Average recall': 0.99285988765695821}
Test scores:  {'Average f_score': 0.93179808093689487, 'Average precision': 0.93355550588566649, 'Average recall': 0.94402061506120105}


In [13]:
test_metrics_df

,Category,f_score,precision,recall
1,related,0.789195,0.788923,0.801495
2,request,0.870159,0.874608,0.882972
3,offer,0.994053,0.992082,0.996033
4,aid_related,0.736722,0.744104,0.743363
5,medical_help,0.893874,0.901953,0.920354
6,medical_products,0.933835,0.936515,0.949802
7,search_and_rescue,0.962659,0.95966,0.974214
8,security,0.974864,0.972096,0.982606
9,military,0.961923,0.960935,0.971315
10,child_alone,1,1,1


In [14]:
train_metrics_df

,Category,f_score,precision,recall
1,related,0.989172,0.989212,0.989218
2,request,0.986319,0.986662,0.986522
3,offer,0.998264,0.998426,0.998423
4,aid_related,0.984707,0.984962,0.984742
5,medical_help,0.986828,0.987484,0.987336
6,medical_products,0.99223,0.992582,0.992524
7,search_and_rescue,0.993143,0.993568,0.993541
8,security,0.994459,0.99487,0.994863
9,military,0.995045,0.995243,0.995219
10,child_alone,1,1,1


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipe.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f418fe16730>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [17]:
parameters = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__n_estimators': [10, 50]}

cv = GridSearchCV(pipe, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
cv.fit(X_train, Y_train)
y_cv_test_preds = cv.predict(X_test)
cv_test_metrics_df, cv_test_avg_metrics = get_metrics(y_test = Y_test, y_pred = y_cv_test_preds)

In [19]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [22]:
cv.get_params

<bound method BaseEstimator.get_params of GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [10, 50, None], 'clf__estimator__n_estimators': [10, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)>

In [21]:
cv_test_avg_metrics

{'Average f_score': 0.93583476938108212,
 'Average precision': 0.93827968025819986,
 'Average recall': 0.94777998169057098}

In [20]:
cv_test_metrics_df

,Category,f_score,precision,recall
1,related,0.802578,0.810391,0.82072
2,request,0.886051,0.891313,0.896399
3,offer,0.994053,0.992082,0.996033
4,aid_related,0.773131,0.774049,0.775252
5,medical_help,0.886998,0.890659,0.916997
6,medical_products,0.931122,0.941234,0.950107
7,search_and_rescue,0.963542,0.966327,0.974825
8,security,0.974581,0.970526,0.982606
9,military,0.960413,0.962871,0.971773
10,child_alone,1,1,1


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### Try other classification algorithms

In [15]:
# Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
pipe_decision_tree = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)
np.random.seed(2021)
pipe_decision_tree.fit(X_train, Y_train)
y_test_preds = pipe_decision_tree.predict(X_test)
test_metrics_df, test_avg_metrics = get_metrics(y_test = Y_test, y_pred = y_test_preds)
print('Decision tree test scores: ', test_avg_metrics)

Decision tree test scores:  {'Average f_score': 0.93088467161949084, 'Average precision': 0.9298058229184174, 'Average recall': 0.93213220086122106}


In [17]:
# K Neighbors classifier
from sklearn.neighbors import KNeighborsClassifier
pipe_knn = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)
np.random.seed(2021)
pipe_knn.fit(X_train, Y_train)
y_test_preds = pipe_knn.predict(X_test)
test_metrics_df, test_avg_metrics = get_metrics(y_test = Y_test, y_pred = y_test_preds)
print('KNeighborsClassifier test scores: ', test_avg_metrics)

KNeighborsClassifier test scores:  {'Average f_score': 0.91407072502879727, 'Average precision': 0.92223341660787639, 'Average recall': 0.93309005526735178}


Best model is the cv Random Forest Classifier. 

### 9. Export your model as a pickle file

In [23]:
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.